# Introduction to Toloka and Toloka API

Toloka is a crowdsourcing platform that helps to analyze large volumes of data in a short period of time.

Examples of common tasks:
* Group the wide variety of items in your online store into categories.
* Find or verify information.
* Translate texts.

[Toloka-Kit](https://github.com/Toloka/toloka-kit) is an open-source library, integrated into Toloka API functionality.

**Useful links:**

- [Toloka Kit documentation](https://toloka.ai/en/docs/toloka-kit/?utm_source=github&utm_medium=site&utm_campaign=tolokakit)
- [Toloka homepage](https://toloka.ai/?utm_source=github&utm_medium=site&utm_campaign=tolokakit)
- [Toloka requester's guide](https://toloka.ai/en/docs/?utm_source=github&utm_medium=site&utm_campaign=tolokakit)
- [Toloka API documentation](https://toloka.ai/en/docs/api/?utm_source=github&utm_medium=site&utm_campaign=tolokakit)

The best way to start is to test Toloka web interface by trying out [one of the tutorials](https://toloka.ai/en/docs/guide/concepts/usecases?utm_source=github&utm_medium=site&utm_campaign=tolokakit).

## Registration

1. [Register](https://toloka.ai/en/docs/guide/concepts/access?utm_source=github&utm_medium=site&utm_campaign=tolokakit) in Toloka as a requester.
2. Choose the backend:
  * The [production backend](https://platform.toloka.ai/for-requesters/?utm_source=github&utm_medium=site&utm_campaign=tolokakit) is used by default in this example.
  * The [sandbox backend](https://platform.sandbox.toloka.ai/for-requesters/?utm_source=github&utm_medium=site&utm_campaign=tolokakit) is a testing environment for Toloka. [Learn more](https://toloka.ai/en/docs/guide/concepts/sandbox?utm_source=github&utm_medium=site&utm_campaign=tolokakit).
3. [Add funds](https://toloka.ai/en/docs/guide/concepts/refill?utm_source=github&utm_medium=site&utm_campaign=tolokakit) to your Toloka account, if you're going to use the production version.
4. [Get an OAuth token](https://toloka.ai/en/docs/api/concepts/access#token?utm_source=github&utm_medium=site&utm_campaign=tolokakit) for your version. Go to **Profile** → **Integrations** → **Get OAuth Token**.

<table  align="center">
  <tr><td>
    <img src="./img/OAuth.png"
         alt="OAuth token"  width="1000">
  </td></tr>
  <tr><td align="center">
    <b>Figure 1.</b> How to get an OAuth token.
  </td></tr>
</table>

### Call to action
If you found some bugs or have a new feature idea, don't hesitate to [open a new issue on Github](https://github.com/Toloka/toloka-kit/issues/new/choose).
Like our library and examples? Star [our repo on Github](https://github.com/Toloka/toloka-kit)

## Getting started with Toloka-Kit
Install Toloka-Kit, import the necessary libraries into your Python script and set up logging.

In [ ]:
%%capture
!pip install toloka-kit==1.0.2
!pip install pandas
!pip install ipyplot

import datetime
import time
import logging
import sys
import getpass

import pandas
import ipyplot

import toloka.client as toloka
import toloka.client.project.template_builder as tb

logging.basicConfig(
    format='[%(levelname)s] %(name)s: %(message)s',
    level=logging.INFO,
    stream=sys.stdout,
)

Сreate a Toloka client instance. All API calls will go through it.

In [ ]:
toloka_client = toloka.TolokaClient(getpass.getpass('Enter your OAuth token: '), 'PRODUCTION') # Or switch to 'SANDBOX'
# Lines below check that the OAuth token is correct and print your account's name
print(toloka_client.get_requester())

Cells below can help you learn more about an object or a method you are interested in.

In [ ]:
toloka.TolokaClient?

In [ ]:
toloka.TolokaClient.get_requester?

In [ ]:
toloka.requester.Requester?

## Toloka entities and how to manage them with Toloka-Kit

### Project
A [project](https://toloka.ai/en/docs/guide/concepts/overview?utm_source=github&utm_medium=site&utm_campaign=tolokakit#project) is a top-level object. It contains instructions, task interface settings, input and output data specification, and default quality control rules for this project pools. Projects make it easier for you to post similar tasks in the future, because you don't have to re-configure the interface.

The easier the task, the better the results. If your task contains more than one question, you should divide it into several projects.

In this tutorial you will create a project with tasks that ask Tolokers to specify the type of animal depicted in a photo.

In [ ]:
new_project = toloka.Project(
    public_name='Cat or Dog?',
    public_description='Specify the type of animal depicted in a photo.',
)

The cell above creates an object in your device memory. This is not all, the project must also contain:
* [Input and output data specification](https://toloka.ai/en/docs/guide/concepts/incoming?utm_source=github&utm_medium=site&utm_campaign=tolokakit)
* [Task interface settings](https://toloka.ai/en/docs/guide/concepts/spec?utm_source=github&utm_medium=site&utm_campaign=tolokakit)
* [An instruction](ttps://toloka.ai/en/docs/guide/concepts/instruction?utm_source=github&utm_medium=site&utm_campaign=tolokakit)

**Important:** Several cells below will add changes to the object stored in your device memory. The data will only be sent to the server after calling one of the `toloka_client` methods.

#### Input and output data

The `image` input field contains URLs of images that need to be labeled.

The `result` output field will receive `cat` and `dog` labels.

In [ ]:
input_specification = {'image': toloka.project.UrlSpec()}
output_specification = {'result': toloka.project.StringSpec()}

#### Task interface

The task interface displays the main task elements to Tolokers. It's important because it is how Tolokers see your tasks. If it's too complicated and unclear, the labeling results might be poor.

There are two editors available in Toloka:
* [HTML/CSS/JS editor](https://toloka.ai/en/docs/guide/concepts/spec?utm_source=github&utm_medium=site&utm_campaign=tolokakit#interface-section)
* [Template Builder](https://toloka.ai/en/docs/template-builder/?utm_source=github&utm_medium=site&utm_campaign=tolokakit)

Template Builder configures task interface at the entity level. We recommend it for your projects, especially for the first ones.

The cell below creates a task interface for our project.

In [ ]:
# This component shows images
image_viewer = tb.ImageViewV1(tb.InputData('image'), ratio=[1, 1])

# This component allows to select a label
radio_group_field = tb.RadioGroupFieldV1(
    tb.OutputData('result'),
    [
        tb.fields.GroupFieldOption('cat', 'Cat'),
        tb.fields.GroupFieldOption('dog', 'Dog')
    ],
    validation=tb.RequiredConditionV1(),
)

# Allows to set a width limit when displaying a task
task_width_plugin = tb.TolokaPluginV1(
    'scroll',
    task_width=400,
)

# How performers will see the task
project_interface = toloka.project.TemplateBuilderViewSpec(
    view=tb.ListViewV1([image_viewer, radio_group_field]),
    plugins=[task_width_plugin],
)

# This block assigns task interface and input/output data specification to the project
# Note that this is done via the task specification class
new_project.task_spec = toloka.project.task_spec.TaskSpec(
    input_spec=input_specification,
    output_spec=output_specification,
    view_spec=project_interface,
)

#### Task instructions

The first thing the Tolokers see when they select a task are the [instructions](https://toloka.ai/en/docs/guide/concepts/instruction?utm_source=github&utm_medium=site&utm_campaign=tolokakit) that you wrote. Describe what needs to be done in simple and clear language, and give examples.

Good instructions help the Tolokers complete the task correctly. The clarity and completeness of the instructions affect the response quality and the project rating. Unclear or too complex instructions, on the contrary, will scare off Tolokers.

Create the instructions for your project with the following cell.

In [ ]:
new_project.public_instructions = 'Look at the picture. Determine what is on it: a <b>cat</b> or a <b>dog</b>. Choose the correct option.'

#### Create a project

Now, use `toloka_client` defined at the beginning to create the project.

The data is only sent to the server after calling one of the `toloka_client` methods, the cell below actually creates a project.

In [ ]:
new_project = toloka_client.create_project(new_project)

### Project preview

1. Preview project

2. Go to the project page to make sure the task interface works correctly. To do this, click the link in the output of the cell above.

3. In the upper-right corner of the project page click ![Project actions](./img/drop-down.svg) → **![Preview](./img/preview.svg) Preview**:

<table  align="center">
  <tr><td>
    <img src="./img/project_look.png"
         alt="Project interface"  width="1000">
  </td></tr>
  <tr><td align="center">
    <b>Figure 2.</b> What the project interface might look like.
  </td></tr>
</table>

4. In the upper part of the preview page click **Change input data**, and insert an image URL into the `image` field, then click **Apply**.

<table  align="center">
  <tr><td>
    <img src="./img/task_look.png"
         alt="Task interface"  width="1000">
  </td></tr>
  <tr><td align="center">
    <b>Figure 3.</b> What the task interface might look like and how to insert images in the preview.
  </td></tr>
</table>

5. In the upper part of the preview demonstration click **Instructions**. Make sure the instructions are shown and that they say what you want them to.

6. Select an option in your task. In the lower-left corner of the preview demonstration click **Submit**, then **View responses**. In the appeared result window, check that your results are written in expected format and that the entered data is correct.

<table  align="center">
  <tr><td>
    <img src="./img/results_preview.png"
         alt="Result priview"  width="1000">
  </td></tr>
  <tr><td align="center">
    <b>Figure 4.</b> What the results might look like.
  </td></tr>
</table>

Tips:
* We strongly recommend **checking the task interface and instructions** every time you create a project. This will help you to ensure that the Tolokers will complete the task and that your results will be useful.
* Do a **trial run** with a small amount of data. Make sure that after running the entire pipeline you get the data in the expected format and quality.

### Pool
A [pool](https://toloka.ai/en/docs/guide/concepts/overview?utm_source=github&utm_medium=site&utm_campaign=tolokakit#pool) is a set of tasks that share common pricing, start date, selection of Tolokers, overlap, and quality control configurations. All task in a pool are processed in parallel. One project can have several pools. You can add new tasks to a pool at any time, as well as open or stop it.

The cell below will create a pool as an object in your device memory. You will send it to Toloka with `toloka_client` method a bit later.

In [ ]:
new_pool = toloka.Pool(
    project_id=new_project.id,
    private_name='Pool 1',  # Only you can see this information
    may_contain_adult_content=False,
    reward_per_assignment=0.01,  # Sets the minimum payment amount for one task suite in USD
    assignment_max_duration_seconds=60*5,  # Gives performers 5 minutes to complete one task suite
    will_expire=datetime.datetime.utcnow() + datetime.timedelta(days=365),  # Sets that the pool will close after one year
)

### Overlap

To minimize the risk of getting wrong answers, you can ask several Tolokers to complete the same task. This is called _overlap_.

In this example we set the overlap to `3`. This means that **every** task will be completed by **three** different Tolokers.

In [ ]:
new_pool.defaults = toloka.pool.Pool.Defaults(
    default_overlap_for_new_tasks=3,
    default_overlap_for_new_task_suites=0,
)

### Task suite

A [task suite](https://toloka.ai/en/docs/guide/concepts/overview?utm_source=github&utm_medium=site&utm_campaign=tolokakit#tasks-page) is a set of tasks that are shown on a single page.

An important part of configuring pools is to decide how many tasks will be issued to a Toloker at once. For example, if you set 3 tasks for a task suite, a Toloker will see three images at once on one page.

Note that the `reward_per_assignment` and `assignment_max_duration_seconds` fields in pool settings set the price and time for one **task suite**, not task.

Why you should combine tasks in a task suite:

* To set a more precise price for a single task.
* To calculate a Toloker's skill and use it to determine the correct answer more accurately. Learn more in the [Aggregation](#aggregation) section.
* To better apply quality control settings that improve the final quality of the response. Learn more in the [Quality control rules](#quality_control_rules) section.

In [ ]:
new_pool.set_mixer_config(
    real_tasks_count=10,  # The number of tasks per page.
)

### Filters

[Filters](https://toloka.ai/en/docs/guide/concepts/filters?utm_source=github&utm_medium=site&utm_campaign=tolokakit) help you select Tolokers for your project.

There may be different reasons to use filters, for example:

* You require some specific group of Tolokers for your pool.
* You want to exclude a certain group of Tolokers.

Tasks will only be shown to matching Tolokers, rather than to all of them.

This example requires English-speaking Tolokers, because the project instructions are in English.

In [ ]:
new_pool.filter = toloka.filter.Languages.in_('EN')

### Quality control rules<a id='quality_control_rules'></a>

[Quality control rules](https://toloka.ai/en/docs/guide/concepts/check-performers?utm_source=github&utm_medium=site&utm_campaign=tolokakit) regulate task completion and Toloker access.

Quality control lets you get more accurate responses and restrict access to tasks for cheating users. All rules work independently. Learn more about [settting up quality control](https://toloka.ai/en/docs/guide/concepts/qa-pool-settings?utm_source=github&utm_medium=site&utm_campaign=tolokakit).

This example uses the captcha rule. It is the simplest way to exclude fake users (robots) and cheaters.

In [ ]:
# Turns on captchas
new_pool.set_captcha_frequency('MEDIUM')
# Bans performers by captcha criteria
new_pool.quality_control.add_action(
    # Type of quality control rule
    collector=toloka.collectors.Captcha(history_size=5),
    # This condition triggers the action below
    # Here overridden comparison operator actually returns a Condition object
    conditions=[toloka.conditions.FailRate > 20],
    # What exactly should the rule do when the condition is met
    # It bans the performer for 1 day
    action=toloka.actions.RestrictionV2(
        scope='PROJECT',
        duration=1,
        duration_unit='DAYS',
        private_comment='Captcha',
    )
)

### Create pool

The cell below creates a pool with all the information above which was stored in your device memory.

In [ ]:
new_pool = toloka_client.create_pool(new_pool)

Open your project page. You will see your new pool.

<table  align="center">
  <tr><td>
    <img src="./img/project_with_pool.png"
         alt="Project interface with a pool"  width="1000">
  </td></tr>
  <tr><td align="center">
    <b>Figure 5.</b> Project interface with a pool.
  </td></tr>
</table>

The pool interface looks like this.

<table  align="center">
  <tr><td>
    <img src="./img/pool_preview.png"
         alt="Pool interface"  width="1000">
  </td></tr>
  <tr><td align="center">
    <b>Figure 6.</b> Pool interface.
  </td></tr>
</table>

Right now the pool is empty and closed. It has no tasks or task suites.

### Upload tasks

A [task](https://toloka.ai/en/docs/guide/concepts/overview?utm_source=github&utm_medium=site&utm_campaign=tolokakit#task) is the smallest portion of data you need to mark up.

This example uses a small data set with images. This dataset is collected by the Toloka team and distributed under a
[![License: CC BY 4.0](https://img.shields.io/badge/License-CC%20BY%204.0-lightgrey.svg)](https://creativecommons.org/licenses/by/4.0/)

In [ ]:
# Download the data set
!curl https://tlk.s3.yandex.net/dataset/cats_vs_dogs/toy_dataset.tsv --output dataset.tsv

dataset = pandas.read_csv('dataset.tsv', sep='\t')

print(f'Dataset contains {len(dataset)} rows\n')

dataset = dataset.sample(frac=1).reset_index(drop=True)

ipyplot.plot_images(
    images=[row['url'] for _, row in dataset.iterrows()],
    labels=[row['label'] for _, row in dataset.iterrows()],
    max_images=12,
    img_width=300,
)

Create tasks. One task will be created from one image.

Toloka will automatically create task suites and show the tasks depending on a project overlap:

1. One task suite will consist of 10 tasks.
2. Toloka will let 3 different Tolokers to complete the tasks.

We configured these settings while creating the pool.

In [ ]:
tasks = [
    toloka.Task(input_values={'image': url}, pool_id=new_pool.id)
    for url in dataset['url']
]
# Add tasks to a pool
toloka_client.create_tasks(tasks, allow_defaults=True)
print(f'Populated pool with {len(tasks)} tasks')
print(f'To view this pool, go to https://toloka.dev/requester/project/{new_project.id}/pool/{new_pool.id}')
# print(f'To view this pool, go to https://sandbox.toloka.dev/requester/project/{new_project.id}/pool/{new_pool.id}') # Print a sandbox version link

# Opens the pool
new_pool = toloka_client.open_pool(new_pool.id)

When you open your pool, Tolokers will see your tasks in their mobile app or in Toloka web version, and start working on them.

In small pools like this, it usually takes up to 10 minutes for all the tasks to be performed.

With big pools, we recommend that you set up automatic waiting. See the example in the cell below.


In [ ]:
pool_id = new_pool.id

def wait_pool_for_close(pool_id, minutes_to_wait=1):
    sleep_time = 60 * minutes_to_wait
    pool = toloka_client.get_pool(pool_id)
    while not pool.is_closed():
        op = toloka_client.get_analytics([toloka.analytics_request.CompletionPercentagePoolAnalytics(subject_id=pool.id)])
        op = toloka_client.wait_operation(op)
        percentage = op.details['value'][0]['result']['value']
        print(
            f'   {datetime.datetime.now().strftime("%H:%M:%S")}\t'
            f'Pool {pool.id} - {percentage}%'
        )
        time.sleep(sleep_time)
        pool = toloka_client.get_pool(pool.id)
    print('Pool was closed.')

wait_pool_for_close(pool_id)

## Get responses

When all the tasks are completed, look at the responses from Tolokers.

In [ ]:
answers_df = toloka_client.get_assignments_df(pool_id)
# Prepare dataframe for aggregation
answers_df = answers_df.rename(columns={
    'INPUT:image': 'task',
    'OUTPUT:result': 'label',
    'ASSIGNMENT:worker_id': 'worker',
})

print(f'answers count: {len(answers_df)}')

An `assignment` value is one Toloker's responses to all the tasks on a task suite.

If a Toloker completed several task suites, then `toloka_client.get_assignments_df` will contain several `assignment` values.

### Aggregation <a id='aggregation'></a>

You should run the [results aggregation](https://toloka.ai/en/docs/guide/concepts/result-aggregation?utm_source=github&utm_medium=site&utm_campaign=tolokakit) only if you set the overlap for the tasks to 2 or higher.

The [majority vote](https://toloka.ai/en/docs/guide/concepts/mvote?utm_source=github&utm_medium=site&utm_campaign=tolokakit) method is a quality control method based on matching responses from the majority of Tolokers who complete the same task. For example, if 2 out of 3 Tolokers selected the `cat` label, then the final label for this task will be `cat`.

Majority vote is easily implemented, but you can also use our crowdsourcing [Crowd-Kit library](https://github.com/Toloka/crowd-kit?utm_source=github&utm_medium=site&utm_campaign=tolokakit). It contains a lot of new aggregation methods.

In [ ]:
!pip install crowd-kit==1.1.0
from crowdkit.aggregation import MajorityVote

In [ ]:
MajorityVote?

In [ ]:
# Run majority vote aggregation
predicted_answers = MajorityVote().fit_predict(answers_df)

print(predicted_answers)

# Some preparations for displaying the results
predicted_answers = predicted_answers.sample(frac=1)
images = predicted_answers.index.values
labels = predicted_answers.values
start_with = 0

Look at the results.

Note: The cell below can be run several times.

In [ ]:
if start_with >= len(predicted_answers):
    print('no more images')
else:
    ipyplot.plot_images(
        images=images[start_with:],
        labels=labels[start_with:],
        max_images=12,
        img_width=300,
    )

    start_with += 12

You can see the labeled images. Some possible results are shown in figure 7 below.

<table  align="center">
  <tr><td>
    <img src="./img/possible_results.png"
         alt="Possible results"  width="1000">
  </td></tr>
  <tr><td align="center">
    <b>Figure 7.</b> Possible results.
  </td></tr>
</table>

## Summary

This example explained basic Toloka entities and how Toloka-Kit can work with them.

The described project (classification) is very useful for:
* Accurate evaluation.
* Checking the results of a complex project, as in [object detection example](https://github.com/Toloka/toloka-kit/tree/main/examples/1.computer_vision/object_detection) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Toloka/toloka-kit/blob/main/examples/1.computer_vision/object_detection/object_detection.ipynb).